## Phase 2: Model Conversion

This notebook converts the custom trained .h5 models into edge-optimized formats.

*   TensorFlow.js format (.json + .bin) for use in a Progressive Web App (PWA)
*   TensorFlow Lite format (.tflite) for use in mobile apps

This ensures models are ready for execution on-device, enabling real-time predictions without server calls.


**1. Setting up the environment**

In [2]:
!pip install -q tensorflow tensorflowjs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 2025.7.1 requires packaging>=24.1, but you have packaging 23.2 which is incompatible.
google-cloud-bigquery 3.35.1 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
db-dtypes 1.4.3 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.


In [29]:
import os
import tensorflow as tf
import tensorflowjs as tfjs
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import json
import os

In [21]:
# define paths
from google.colab import drive
drive.mount('/content/drive')

BASE_PATH = "/content/drive/My Drive/Edge_AI_Project/models"
H5_PATH = os.path.join(BASE_PATH, "saved_models")
TFJS_PATH = os.path.join(BASE_PATH, "tfjs_models")
TFLITE_PATH = os.path.join(BASE_PATH, "tflite_models")
LOG_PATH = os.path.join(BASE_PATH, "conversion_logs")

# Clean up and recreate directories
import shutil

# Remove existing converted models to start fresh
if os.path.exists(TFJS_PATH):
    shutil.rmtree(TFJS_PATH)
if os.path.exists(TFLITE_PATH):
    shutil.rmtree(TFLITE_PATH)

os.makedirs(TFJS_PATH, exist_ok=True)
os.makedirs(TFLITE_PATH, exist_ok=True)
os.makedirs(LOG_PATH, exist_ok=True)

print("✅ Directories cleaned and recreated")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Directories cleaned and recreated


**2. Model Conversion Functions**

In [22]:
def convert_to_tfjs_simple(h5_model_path, output_dir, model_name):
    """
    Simple TensorFlow.js conversion without model reconstruction
    """
    print(f"\n🔄 Converting {model_name} to TensorFlow.js...")

    # Load original model
    model = load_model(h5_model_path)
    print(f"   Loaded model: {model.input_shape} -> {model.output_shape}")

    # Test the model works
    if model.input_shape[1:] == (224, 224, 3):
        test_input = np.random.random((1, 224, 224, 3)).astype(np.float32)
    elif model.input_shape[1:] == (48, 48, 1):
        test_input = np.random.random((1, 48, 48, 1)).astype(np.float32)
    else:
        raise ValueError(f"Unexpected input shape: {model.input_shape}")

    test_output = model.predict(test_input, verbose=0)
    print(f"   Model test successful - Output shape: {test_output.shape}")

    # Convert to TensorFlow.js with UPDATED API
    print(f"   Converting to TensorFlow.js format...")
    tfjs.converters.save_keras_model(model, output_dir)

    print(f"✅ Successfully converted {model_name} to TensorFlow.js")

    # Verify the conversion
    model_files = os.listdir(output_dir)
    print(f"   Generated files: {model_files}")

    return model

def convert_to_tflite_simple(h5_model_path, output_path, model_name):
    """
    Simple TensorFlow Lite conversion
    """
    print(f"\n🔄 Converting {model_name} to TensorFlow Lite...")

    model = load_model(h5_model_path)
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Post-training quantization
    tflite_model = converter.convert()

    with open(output_path, 'wb') as f:
        f.write(tflite_model)

    print(f"✅ Successfully converted {model_name} to TensorFlow Lite")


**3. Model configuration**

In [23]:
models = {
    "age": {
        "filename": "age_model.h5",
        "input_shape": (224, 224, 3)
    },
    "gender": {
        "filename": "gender_model.h5",
        "input_shape": (224, 224, 3)
    },
    "emotion": {
        "filename": "emotion_model.h5",
        "input_shape": (48, 48, 1)
    }
}

**4. Conversion and Logging Loop**

In [24]:
log_data = []

print("Starting model conversion process...\n")

for name, config in models.items():
    print(f"{'='*60}")
    print(f"🔄 Processing {name.upper()} model...")
    print(f"{'='*60}")

    h5_path = os.path.join(H5_PATH, config["filename"])
    tfjs_dir = os.path.join(TFJS_PATH, name)
    tflite_path = os.path.join(TFLITE_PATH, f"{name}.tflite")

    try:
        # Convert to TensorFlow.js
        model = convert_to_tfjs_simple(h5_path, tfjs_dir, name)

        # Convert to TFLite
        convert_to_tflite_simple(h5_path, tflite_path, name)

        # Calculate file sizes
        tfjs_size = sum(os.path.getsize(os.path.join(tfjs_dir, f)) for f in os.listdir(tfjs_dir))
        tflite_size = os.path.getsize(tflite_path)
        h5_size = os.path.getsize(h5_path)

        # Log the results
        log_data.append({
            "Model": name.capitalize(),
            "Input Shape": config["input_shape"],
            ".h5 Size (KB)": round(h5_size / 1024, 2),
            "TF.js Size (KB)": round(tfjs_size / 1024, 2),
            ".tflite Size (KB)": round(tflite_size / 1024, 2),
            "Model Layers": len(model.layers),
            "Output .tflite": tflite_path,
            "Output TF.js Dir": tfjs_dir
        })

        print(f"✅ {name.upper()} model conversion completed successfully!\n")

    except Exception as e:
        print(f"❌ Error processing {name} model: {str(e)}")
        import traceback
        traceback.print_exc()

print("\n" + "="*70)
print("Model conversion completed!")
print("="*70)

Starting model conversion process...

🔄 Processing AGE model...

🔄 Converting age to TensorFlow.js...


   Loaded model: (None, 224, 224, 3) -> (None, 1)


   Model test successful - Output shape: (1, 1)
   Converting to TensorFlow.js format...
failed to lookup keras version from the file,
    this is likely a weight only file
✅ Successfully converted age to TensorFlow.js
   Generated files: ['group1-shard1of3.bin', 'group1-shard2of3.bin', 'group1-shard3of3.bin', 'model.json']

🔄 Converting age to TensorFlow Lite...


Saved artifact at '/tmp/tmpvrsb2xmo'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  132901495619408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901495616720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901495616528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901495616912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901495618448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901495614032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901495613456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901495613072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901495614800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901495617680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901495614992

   Loaded model: (None, 224, 224, 3) -> (None, 1)


   Model test successful - Output shape: (1, 1)
   Converting to TensorFlow.js format...
failed to lookup keras version from the file,
    this is likely a weight only file
✅ Successfully converted gender to TensorFlow.js
   Generated files: ['group1-shard1of3.bin', 'group1-shard2of3.bin', 'group1-shard3of3.bin', 'model.json']

🔄 Converting gender to TensorFlow Lite...


Saved artifact at '/tmp/tmppmrjkswi'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  132901459148240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901459142672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901459148048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901459147088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901459148432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901459148624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901457576976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901457576016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901459145936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901457576208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901457578320

✅ Successfully converted gender to TensorFlow Lite
✅ GENDER model conversion completed successfully!

🔄 Processing EMOTION model...

🔄 Converting emotion to TensorFlow.js...
   Loaded model: (None, 48, 48, 1) -> (None, 1)


   Model test successful - Output shape: (1, 1)
   Converting to TensorFlow.js format...
failed to lookup keras version from the file,
    this is likely a weight only file
✅ Successfully converted emotion to TensorFlow.js
   Generated files: ['group1-shard1of3.bin', 'group1-shard2of3.bin', 'group1-shard3of3.bin', 'model.json']

🔄 Converting emotion to TensorFlow Lite...


Saved artifact at '/tmp/tmp397eflyp'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 48, 48, 1), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  132901462973648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901462982096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901482778256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901482788048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901482784208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901482787472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901482778832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901482790544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901482782672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901482783632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132901482786320

**5. Save conversion logs**

In [25]:
if log_data:
    df = pd.DataFrame(log_data)
    log_file = os.path.join(LOG_PATH, "simple_conversion_summary.csv")
    df.to_csv(log_file, index=False)

    print(f"\n📊 Conversion Summary:")
    print(df.to_string(index=False))
    print(f"\n💾 Log saved to: {log_file}")




📊 Conversion Summary:
  Model   Input Shape  .h5 Size (KB)  TF.js Size (KB)  .tflite Size (KB)  Model Layers                                                              Output .tflite                                                   Output TF.js Dir
    Age (224, 224, 3)       26885.20          8979.97            2443.16           156     /content/drive/My Drive/Edge_AI_Project/models/tflite_models/age.tflite     /content/drive/My Drive/Edge_AI_Project/models/tfjs_models/age
 Gender (224, 224, 3)       26885.20          8979.97            2443.16           156  /content/drive/My Drive/Edge_AI_Project/models/tflite_models/gender.tflite  /content/drive/My Drive/Edge_AI_Project/models/tfjs_models/gender
Emotion   (48, 48, 1)       25666.21          8548.44            2142.49             9 /content/drive/My Drive/Edge_AI_Project/models/tflite_models/emotion.tflite /content/drive/My Drive/Edge_AI_Project/models/tfjs_models/emotion

💾 Log saved to: /content/drive/My Drive/Edge_AI_Project/

**6. Verification**

In [26]:
print(f"\n Verification:")
print("Checking if model.json files were created correctly...")

for name in models.keys():
    tfjs_dir = os.path.join(TFJS_PATH, name)
    model_json_path = os.path.join(tfjs_dir, "model.json")

    if os.path.exists(model_json_path):
        print(f"   ✅ {name}: model.json exists")

        # Check file size
        json_size = os.path.getsize(model_json_path)
        if json_size > 1000:  # Should be at least 1KB
            print(f"   ✅ {name}: model.json has reasonable size ({json_size} bytes)")
        else:
            print(f"   ⚠️  {name}: model.json seems too small ({json_size} bytes)")
    else:
        print(f"   ❌ {name}: model.json not found")

print(f"\n Verification CompleteE!")


 Verification:
Checking if model.json files were created correctly...
   ✅ age: model.json exists
   ✅ age: model.json has reasonable size (158430 bytes)
   ✅ gender: model.json exists
   ✅ gender: model.json has reasonable size (158430 bytes)
   ✅ emotion: model.json exists
   ✅ emotion: model.json has reasonable size (9148 bytes)

 Verification CompleteE!


Resolving issue with input shape information

In [36]:
# Add missing input shape information that TensorFlow.js needs

# FINAL DEFINITIVE FIX - Replace Python None with JSON null
import json
import os

BASE_PATH = "/content/drive/My Drive/Edge_AI_Project/models/tfjs_models"

def fix_none_to_null(model_name):
    """
    Replace all Python None values with JSON null values
    """
    model_json_path = os.path.join(BASE_PATH, model_name, 'model.json')

    print(f"🔧 Fixing {model_name} - Converting None to null...")

    # Read the file as text first
    with open(model_json_path, 'r') as f:
        content = f.read()

    # Count None occurrences before fix
    none_count = content.count('None')
    print(f"   Found {none_count} instances of 'None'")

    # Replace Python None with JSON null
    fixed_content = content.replace('None', 'null')

    # Also fix Python booleans if they exist
    fixed_content = fixed_content.replace('True', 'true')
    fixed_content = fixed_content.replace('False', 'false')

    # Write back the fixed content
    with open(model_json_path, 'w') as f:
        f.write(fixed_content)

    # Verify the fix
    with open(model_json_path, 'r') as f:
        new_content = f.read()

    none_count_after = new_content.count('None')
    null_count_after = new_content.count('null')

    print(f"   ✅ After fix: {none_count_after} 'None', {null_count_after} 'null'")

    # Verify it's valid JSON
    try:
        with open(model_json_path, 'r') as f:
            json.load(f)
        print(f"   ✅ Valid JSON format confirmed")
        return True
    except json.JSONDecodeError as e:
        print(f"   ❌ JSON validation failed: {e}")
        return False

print("🚀 FINAL FIX: Converting Python None to JSON null")
print("="*60)

success_count = 0
for model_name in ['age', 'gender', 'emotion']:
    if fix_none_to_null(model_name):
        success_count += 1
    print()

print(f"🎉 COMPLETED: {success_count}/3 models fixed successfully!")

# Final verification
print("\n🔍 FINAL VERIFICATION:")
for model_name in ['age', 'gender', 'emotion']:
    model_json_path = os.path.join(BASE_PATH, model_name, 'model.json')

    with open(model_json_path, 'r') as f:
        content = f.read()

    none_count = content.count('None')
    null_count = content.count('null')

    if none_count == 0 and null_count > 0:
        print(f"   ✅ {model_name}: Clean JSON (0 'None', {null_count} 'null')")
    else:
        print(f"   ❌ {model_name}: Still has issues ({none_count} 'None', {null_count} 'null')")

print(f"\n✅ All models should now have proper JSON null values!")
print(f"📥 Download these fixed models and replace your GitHub files.")

🚀 FINAL FIX: Converting Python None to JSON null
🔧 Fixing age - Converting None to null...
   Found 0 instances of 'None'
   ✅ After fix: 0 'None', 1222 'null'
   ✅ Valid JSON format confirmed

🔧 Fixing gender - Converting None to null...
   Found 0 instances of 'None'
   ✅ After fix: 0 'None', 1222 'null'
   ✅ Valid JSON format confirmed

🔧 Fixing emotion - Converting None to null...
   Found 0 instances of 'None'
   ✅ After fix: 0 'None', 75 'null'
   ✅ Valid JSON format confirmed

🎉 COMPLETED: 3/3 models fixed successfully!

🔍 FINAL VERIFICATION:
   ✅ age: Clean JSON (0 'None', 1222 'null')
   ✅ gender: Clean JSON (0 'None', 1222 'null')
   ✅ emotion: Clean JSON (0 'None', 75 'null')

✅ All models should now have proper JSON null values!
📥 Download these fixed models and replace your GitHub files.


In [34]:
# DIAGNOSTIC SCRIPT - Let's see the actual JSON structure
import json
import os

BASE_PATH = "/content/drive/My Drive/Edge_AI_Project/models/tfjs_models"

def diagnose_model_json(model_name):
    """
    Examine the actual structure of the model.json file
    """
    model_json_path = os.path.join(BASE_PATH, model_name, 'model.json')

    print(f"\n{'='*50}")
    print(f"🔍 DIAGNOSING {model_name.upper()} MODEL")
    print(f"{'='*50}")

    if not os.path.exists(model_json_path):
        print(f"❌ File not found: {model_json_path}")
        return

    with open(model_json_path, 'r') as f:
        model_config = json.load(f)

    print(f"📁 File exists: {model_json_path}")
    print(f"📊 File size: {os.path.getsize(model_json_path)} bytes")

    # Show top-level structure
    print(f"\n🏗️  TOP-LEVEL STRUCTURE:")
    for key in model_config.keys():
        print(f"   - {key}")

    # Examine modelTopology structure
    if 'modelTopology' in model_config:
        print(f"\n🧠 MODEL TOPOLOGY STRUCTURE:")
        topology = model_config['modelTopology']
        for key in topology.keys():
            print(f"   - {key}")

        # Look for model_config
        if 'model_config' in topology:
            print(f"\n⚙️  MODEL CONFIG STRUCTURE:")
            model_cfg = topology['model_config']
            for key in model_cfg.keys():
                print(f"   - {key}")

            # Look for layers
            if 'config' in model_cfg and 'layers' in model_cfg['config']:
                layers = model_cfg['config']['layers']
                print(f"\n🔗 LAYERS FOUND: {len(layers)} layers")

                # Show first few layers
                for i, layer in enumerate(layers[:3]):
                    print(f"\n   Layer {i}: {layer.get('class_name', 'Unknown')}")
                    print(f"   Config keys: {list(layer.get('config', {}).keys())}")

                    # Check for input shape info
                    config = layer.get('config', {})
                    for shape_key in ['batch_input_shape', 'input_shape', 'shape']:
                        if shape_key in config:
                            print(f"   ✅ Found {shape_key}: {config[shape_key]}")
                        else:
                            print(f"   ❌ Missing {shape_key}")
            else:
                print(f"❌ No layers found in expected location")
    else:
        print(f"❌ No modelTopology found")

    # Show a small sample of the raw JSON
    print(f"\n📝 FIRST 500 CHARACTERS OF JSON:")
    with open(model_json_path, 'r') as f:
        content = f.read()
        print(content[:500] + "...")

# Diagnose all models
models = ['age', 'gender', 'emotion']

for model_name in models:
    diagnose_model_json(model_name)

print(f"\n" + "="*70)
print(f"🎯 DIAGNOSTIC COMPLETE")
print(f"="*70)
print(f"This will show us the EXACT structure so we can fix it properly!")


🔍 DIAGNOSING AGE MODEL
📁 File exists: /content/drive/My Drive/Edge_AI_Project/models/tfjs_models/age/model.json
📊 File size: 356570 bytes

🏗️  TOP-LEVEL STRUCTURE:
   - format
   - generatedBy
   - convertedBy
   - modelTopology
   - weightsManifest

🧠 MODEL TOPOLOGY STRUCTURE:
   - keras_version
   - backend
   - model_config
   - training_config

⚙️  MODEL CONFIG STRUCTURE:
   - class_name
   - config

🔗 LAYERS FOUND: 156 layers

   Layer 0: InputLayer
   Config keys: ['batch_shape', 'dtype', 'sparse', 'name', 'batch_input_shape']
   ✅ Found batch_input_shape: [None, 224, 224, 3]
   ❌ Missing input_shape
   ❌ Missing shape

   Layer 1: Conv2D
   Config keys: ['name', 'trainable', 'dtype', 'filters', 'kernel_size', 'strides', 'padding', 'data_format', 'dilation_rate', 'groups', 'activation', 'use_bias', 'kernel_initializer', 'bias_initializer', 'kernel_regularizer', 'bias_regularizer', 'activity_regularizer', 'kernel_constraint', 'bias_constraint']
   ❌ Missing batch_input_shape
   ❌

In [35]:
# Simple verification - just check if TensorFlow.js can read the models
import json
import os

BASE_PATH = "/content/drive/My Drive/Edge_AI_Project/models/tfjs_models"

for model_name in ['age', 'gender', 'emotion']:
    model_json_path = os.path.join(BASE_PATH, model_name, 'model.json')

    with open(model_json_path, 'r') as f:
        model_config = json.load(f)

    # Check the first layer
    first_layer = model_config['modelTopology']['model_config']['config']['layers'][0]
    batch_shape = first_layer['config']['batch_input_shape']

    print(f"✅ {model_name}: {batch_shape}")

✅ age: [None, 224, 224, 3]
✅ gender: [None, 224, 224, 3]
✅ emotion: [None, 48, 48, 1]


In [32]:
#  Download the newly converted models
import shutil
import os

# Zip the converted models for easy download
shutil.make_archive('/content/fixed_tfjs_models', 'zip', '/content/drive/My Drive/Edge_AI_Project/models/tfjs_models')

print("✅ Models zipped! Download from:")
print("/content/fixed_tfjs_models.zip")

✅ Models zipped! Download from:
/content/fixed_tfjs_models.zip
